In [1]:
import pandas as pd
import json
import re
import math

In [4]:
df= pd.read_json('../data/seven_data.json')
df

ValueError: Expected object or value

In [3]:
#商品名が怪しくなっている部分があるがどこかわからないので保留(ざっくり確認したが問題なし)
for i in range(0,len(df)):
    data = df['商品名'][i][0]
    df['商品名'][i] = data.replace('[','').replace(']','')
df['商品名']

NameError: name 'df' is not defined

In [4]:
df['値段'] = df['値段'].str.replace('税込','').str.replace('円','')

In [5]:
result = []
result2 = []
for i in range(0,len(df)):
    tmp = re.sub('（[0-9]+.[0-9]+）', "", df['値段'][i])
    tmp2 = int(tmp)*1.08
    tmp2 = math.floor(tmp2)
    result.append(tmp)
    result2.append(tmp2)
_df = pd.DataFrame(result, columns=['税抜き'])
df = df.join(_df)
_df = pd.DataFrame(result2, columns=['税込み'])
df = df.join(_df)

In [6]:
a = df['栄養成分']
a = a.str.extract(r'(?P<カロリー>熱量：[0-9]+kcal)', expand=True)
df = df.join(a)

df['カロリー'] = df['カロリー'].str.replace('熱量：','').str.replace('kcal','')


In [7]:
df = df.astype({'税込み': float}).astype({'税抜き': float}).astype({'カロリー': float})

In [8]:
'''
カロリーフラグ
0~99:0
100~199:1
200~299:2

'''
df = df.assign(カロリーフラグ="")
for x in range(0,len(df['カロリー'])):
    if df['カロリー'][x]<100:
        df['カロリーフラグ'][x] = 0
    elif 100<=df['カロリー'][x] and df['カロリー'][x]<200:
        df['カロリーフラグ'][x] = 1
    elif 200<=df['カロリー'][x] and df['カロリー'][x]<300:
        df['カロリーフラグ'][x] = 2
    elif 300<=df['カロリー'][x] and df['カロリー'][x]<400:
        df['カロリーフラグ'][x] = 3
    elif 400<=df['カロリー'][x] and df['カロリー'][x]<500:
        df['カロリーフラグ'][x] = 4
    elif 500<=df['カロリー'][x] and df['カロリー'][x]<600:
        df['カロリーフラグ'][x] = 5
    elif 600<=df['カロリー'][x] and df['カロリー'][x]<700:
        df['カロリーフラグ'][x] = 6
    elif 700<=df['カロリー'][x] and df['カロリー'][x]<800:
        df['カロリーフラグ'][x] = 7
    elif 800<=df['カロリー'][x] and df['カロリー'][x]<900:
        df['カロリーフラグ'][x] = 8
    elif 900<=df['カロリー'][x] and df['カロリー'][x]<1000:
        df['カロリーフラグ'][x] = 9
        
_

/tmp/ipykernel_364/1417407080.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['カロリーフラグ'][x] = 3
/tmp/ipykernel_364/1417407080.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['カロリーフラグ'][x] = 2
/tmp/ipykernel_364/1417407080.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['カロリーフラグ'][x] = 4
/tmp/ipykernel_364/1417407080.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

0                  ももももサンド
1                  ももももサンド
2       シャインマスカット入りフルーツサンド
3           大きなソーセージ　チリドッグ
4              てりたまチーズバーガー
               ...        
1493     ７プレミアムゼスプリグリーンキウイ
1494           ちょこっとカットパイン
1495            ７プレミアム干しいも
1496    ７プレミアム　けんちん汁・豚汁用水煮
1497         ７プレミアム　さといも水煮
Name: 商品名, Length: 1498, dtype: object

In [9]:
df.to_json('seven_data_shaped.json',force_ascii=False)

In [9]:
df= pd.read_json('../data/seven_data_shaped.json')
df

,商品名,値段,説明,本製品に含まれるアレルギー物質,栄養成分,URL,店名,主要原料・主要原料原産地,原産国,税抜き,税込み,カロリー,カロリーフラグ
0,ももももサンド,340（367.20）,カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：320kcal、たんぱく質：4.1g、脂質：17.3g、炭水化物：37.9g（糖質：3...,https://www.sej.co.jp/products/a/item/051550/,セブンイレブン,None,None,340,367,320.0,3
1,ももももサンド,340（367.20）,カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：306kcal、たんぱく質：4.1g、脂質：16.0g、炭水化物：37.4g（糖質：3...,https://www.sej.co.jp/products/a/item/051545/,セブンイレブン,None,None,340,367,306.0,3
2,シャインマスカット入りフルーツサンド,390（421.20）,９月～１０月に旬を迎えるマスカットを使用したフルーツサンド。他にもパイン、黄桃、みかんの３種...,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：260kcal、たんぱく質：3.5g、脂質：12.8g、炭水化物：33.4g（糖質：3...,https://www.sej.co.jp/products/a/item/050516/,セブンイレブン,None,None,390,421,260.0,2
3,大きなソーセージ チリドッグ,310（334.80）,パリッとジューシーなソーセージにスパイシーなチリソースとチーズを盛り付けました。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：478kcal、たんぱく質：18.8g、脂質：34.0g、炭水化物：25.1g（糖質：...,https://www.sej.co.jp/products/a/item/051402/,セブンイレブン,None,None,310,334,478.0,4
4,てりたまチーズバーガー,380（410.40）,ビーフハンバーグと潰し目玉焼きとチェダーチーズを組み合わせ、照焼ソースとオーロラソースの２種...,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：511kcal、たんぱく質：24.1g、脂質：25.6g、炭水化物：47.9g（糖質：...,https://www.sej.co.jp/products/a/item/051555/,セブンイレブン,None,None,380,410,511.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,７プレミアムゼスプリグリーンキウイ,188（203.04）,,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：37kcal、たんぱく質：0.6g、脂質：0.4g、炭水化物：9.3g（糖質：7.7g...,https://www.sej.co.jp/products/a/item/250998/,セブンイレブン,None,None,188,203,37.0,0
1494,ちょこっとカットパイン,195（210.60）,,None,None,https://www.sej.co.jp/products/a/item/251026/,セブンイレブン,None,None,195,210,NaN,
1495,７プレミアム干しいも,338（365.04）,ほど良い食感と素朴な甘さが特長の干しいもです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：260kcal、たんぱく質：3.9g、脂質：0.5g、炭水化物：62.4g、食塩相当量...,https://www.sej.co.jp/products/a/item/250735/,セブンイレブン,None,None,338,365,260.0,2
1496,７プレミアム けんちん汁・豚汁用水煮,278（300.24）,下ごしらえいらず 家庭の味をちょっとお手伝い,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：26kcal、たんぱく質：0.6g、脂質：0.1g、炭水化物：6.1g（糖質：3.4g...,https://www.sej.co.jp/products/a/item/250594/,セブンイレブン,None,None,278,300,26.0,0


In [10]:
df = df.assign(カロリーフラグ20="")
for x in range(0,len(df['カロリー'])):
    if df['カロリー'][x]<20:
        df['カロリーフラグ20'][x] = 0
    elif 20<=df['カロリー'][x] and df['カロリー'][x]<40:
        df['カロリーフラグ20'][x] = 1
    elif 40<=df['カロリー'][x] and df['カロリー'][x]<60:
        df['カロリーフラグ20'][x] = 2
    elif 60<=df['カロリー'][x] and df['カロリー'][x]<80:
        df['カロリーフラグ20'][x] = 3
    elif 80<=df['カロリー'][x] and df['カロリー'][x]<100:
        df['カロリーフラグ20'][x] = 4
    
    elif 100<=df['カロリー'][x] and df['カロリー'][x]<120:
        df['カロリーフラグ20'][x] = 5
    elif 120<=df['カロリー'][x] and df['カロリー'][x]<140:
        df['カロリーフラグ20'][x] = 6
    elif 140<=df['カロリー'][x] and df['カロリー'][x]<160:
        df['カロリーフラグ20'][x] = 7
    elif 160<=df['カロリー'][x] and df['カロリー'][x]<180:
        df['カロリーフラグ20'][x] = 8
    elif 180<=df['カロリー'][x] and df['カロリー'][x]<200:
        df['カロリーフラグ20'][x] = 9
    
    elif 200<=df['カロリー'][x] and df['カロリー'][x]<220:
        df['カロリーフラグ20'][x] = 10
    elif 220<=df['カロリー'][x] and df['カロリー'][x]<240:
        df['カロリーフラグ20'][x] = 11
    elif 240<=df['カロリー'][x] and df['カロリー'][x]<260:
        df['カロリーフラグ20'][x] = 12
    elif 260<=df['カロリー'][x] and df['カロリー'][x]<280:
        df['カロリーフラグ20'][x] = 13
    elif 280<=df['カロリー'][x] and df['カロリー'][x]<300:
        df['カロリーフラグ20'][x] = 14

    elif 300<=df['カロリー'][x] and df['カロリー'][x]<320:
        df['カロリーフラグ20'][x] = 15
    elif 320<=df['カロリー'][x] and df['カロリー'][x]<340:
        df['カロリーフラグ20'][x] = 16
    elif 340<=df['カロリー'][x] and df['カロリー'][x]<360:
        df['カロリーフラグ20'][x] = 17
    elif 360<=df['カロリー'][x] and df['カロリー'][x]<380:
        df['カロリーフラグ20'][x] = 18
    elif 380<=df['カロリー'][x] and df['カロリー'][x]<400:
        df['カロリーフラグ20'][x] = 19

    elif 400<=df['カロリー'][x] and df['カロリー'][x]<420:
        df['カロリーフラグ20'][x] = 20
    elif 420<=df['カロリー'][x] and df['カロリー'][x]<440:
        df['カロリーフラグ20'][x] = 21
    elif 440<=df['カロリー'][x] and df['カロリー'][x]<460:
        df['カロリーフラグ20'][x] = 22
    elif 460<=df['カロリー'][x] and df['カロリー'][x]<480:
        df['カロリーフラグ20'][x] = 23
    elif 480<=df['カロリー'][x] and df['カロリー'][x]<500:
        df['カロリーフラグ20'][x] = 24

    elif 500<=df['カロリー'][x] and df['カロリー'][x]<520:
        df['カロリーフラグ20'][x] = 25
    elif 520<=df['カロリー'][x] and df['カロリー'][x]<540:
        df['カロリーフラグ20'][x] = 26
    elif 540<=df['カロリー'][x] and df['カロリー'][x]<560:
        df['カロリーフラグ20'][x] = 27
    elif 560<=df['カロリー'][x] and df['カロリー'][x]<580:
        df['カロリーフラグ20'][x] = 28
    elif 580<=df['カロリー'][x] and df['カロリー'][x]<600:
        df['カロリーフラグ20'][x] = 29

    elif 600<=df['カロリー'][x] and df['カロリー'][x]<620:
        df['カロリーフラグ20'][x] = 30
    elif 620<=df['カロリー'][x] and df['カロリー'][x]<640:
        df['カロリーフラグ20'][x] = 31
    elif 640<=df['カロリー'][x] and df['カロリー'][x]<660:
        df['カロリーフラグ20'][x] = 32
    elif 660<=df['カロリー'][x] and df['カロリー'][x]<680:
        df['カロリーフラグ20'][x] = 33
    elif 680<=df['カロリー'][x] and df['カロリー'][x]<700:
        df['カロリーフラグ20'][x] = 34

    elif 700<=df['カロリー'][x] and df['カロリー'][x]<720:
        df['カロリーフラグ20'][x] = 35
    elif 720<=df['カロリー'][x] and df['カロリー'][x]<740:
        df['カロリーフラグ20'][x] = 36
    elif 740<=df['カロリー'][x] and df['カロリー'][x]<760:
        df['カロリーフラグ20'][x] = 37
    elif 760<=df['カロリー'][x] and df['カロリー'][x]<780:
        df['カロリーフラグ20'][x] = 38
    elif 780<=df['カロリー'][x] and df['カロリー'][x]<800:
        df['カロリーフラグ20'][x] = 39

    elif 800<=df['カロリー'][x] and df['カロリー'][x]<820:
        df['カロリーフラグ20'][x] = 40
    elif 820<=df['カロリー'][x] and df['カロリー'][x]<840:
        df['カロリーフラグ20'][x] = 41
    elif 840<=df['カロリー'][x] and df['カロリー'][x]<860:
        df['カロリーフラグ20'][x] = 42
    elif 860<=df['カロリー'][x] and df['カロリー'][x]<880:
        df['カロリーフラグ20'][x] = 43
    elif 880<=df['カロリー'][x] and df['カロリー'][x]<900:
        df['カロリーフラグ20'][x] = 44

    elif 900<=df['カロリー'][x] and df['カロリー'][x]<920:
        df['カロリーフラグ20'][x] = 45
    elif 920<=df['カロリー'][x] and df['カロリー'][x]<940:
        df['カロリーフラグ20'][x] = 46
    elif 940<=df['カロリー'][x] and df['カロリー'][x]<960:
        df['カロリーフラグ20'][x] = 47
    elif 960<=df['カロリー'][x] and df['カロリー'][x]<980:
        df['カロリーフラグ20'][x] = 48
    elif 980<=df['カロリー'][x] and df['カロリー'][x]<1000:
        df['カロリーフラグ20'][x] = 49

/var/folders/pl/qpfspzh954vc65nmb5g7t0q40000gn/T/ipykernel_9337/1542544770.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['カロリーフラグ20'][x] = 16
/var/folders/pl/qpfspzh954vc65nmb5g7t0q40000gn/T/ipykernel_9337/1542544770.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['カロリーフラグ20'][x] = 15
/var/folders/pl/qpfspzh954vc65nmb5g7t0q40000gn/T/ipykernel_9337/1542544770.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['カロリ

In [12]:
df

,商品名,値段,説明,本製品に含まれるアレルギー物質,栄養成分,URL,店名,主要原料・主要原料原産地,原産国,税抜き,税込み,カロリー,カロリーフラグ,カロリーフラグ20
0,ももももサンド,340（367.20）,カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：320kcal、たんぱく質：4.1g、脂質：17.3g、炭水化物：37.9g（糖質：3...,https://www.sej.co.jp/products/a/item/051550/,セブンイレブン,None,None,340,367,320.0,3,16
1,ももももサンド,340（367.20）,カスタードクリームとホイップクリームを組み合わせ、２種類の桃を使用したサンドイッチです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：306kcal、たんぱく質：4.1g、脂質：16.0g、炭水化物：37.4g（糖質：3...,https://www.sej.co.jp/products/a/item/051545/,セブンイレブン,None,None,340,367,306.0,3,15
2,シャインマスカット入りフルーツサンド,390（421.20）,９月～１０月に旬を迎えるマスカットを使用したフルーツサンド。他にもパイン、黄桃、みかんの３種...,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：260kcal、たんぱく質：3.5g、脂質：12.8g、炭水化物：33.4g（糖質：3...,https://www.sej.co.jp/products/a/item/050516/,セブンイレブン,None,None,390,421,260.0,2,13
3,大きなソーセージ チリドッグ,310（334.80）,パリッとジューシーなソーセージにスパイシーなチリソースとチーズを盛り付けました。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：478kcal、たんぱく質：18.8g、脂質：34.0g、炭水化物：25.1g（糖質：...,https://www.sej.co.jp/products/a/item/051402/,セブンイレブン,None,None,310,334,478.0,4,23
4,てりたまチーズバーガー,380（410.40）,ビーフハンバーグと潰し目玉焼きとチェダーチーズを組み合わせ、照焼ソースとオーロラソースの２種...,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：511kcal、たんぱく質：24.1g、脂質：25.6g、炭水化物：47.9g（糖質：...,https://www.sej.co.jp/products/a/item/051555/,セブンイレブン,None,None,380,410,511.0,5,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,７プレミアムゼスプリグリーンキウイ,188（203.04）,,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：37kcal、たんぱく質：0.6g、脂質：0.4g、炭水化物：9.3g（糖質：7.7g...,https://www.sej.co.jp/products/a/item/250998/,セブンイレブン,None,None,188,203,37.0,0,1
1494,ちょこっとカットパイン,195（210.60）,,None,None,https://www.sej.co.jp/products/a/item/251026/,セブンイレブン,None,None,195,210,NaN,,
1495,７プレミアム干しいも,338（365.04）,ほど良い食感と素朴な甘さが特長の干しいもです。,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：260kcal、たんぱく質：3.9g、脂質：0.5g、炭水化物：62.4g、食塩相当量...,https://www.sej.co.jp/products/a/item/250735/,セブンイレブン,None,None,338,365,260.0,2,13
1496,７プレミアム けんちん汁・豚汁用水煮,278（300.24）,下ごしらえいらず 家庭の味をちょっとお手伝い,\n\t\t\t\t\t\t\t\t\t\t\t特定原材料7品目\n\t\t\t\t\t\t...,熱量：26kcal、たんぱく質：0.6g、脂質：0.1g、炭水化物：6.1g（糖質：3.4g...,https://www.sej.co.jp/products/a/item/250594/,セブンイレブン,None,None,278,300,26.0,0,1


In [13]:
df.to_json('seven_data_shaped20.json',force_ascii=False)